# Word Embeddings para Encontrar Preguntas Similares en StackOverflow

Este proyecto resuelve el problema de encontrar preguntas duplicadas o similares en un foro, aplicado específicamente a los títulos de preguntas en StackOverflow. La estrategia se basa en encontrar textos que son similares semánticamente utilizando word embeddings.

## Contenido

- Encontrar preguntas similares en StackOverflow    
    - [Librerías](#toc2_1_1_)    
    - [Datos](#toc2_1_2_)    
  - [Word embeddings](#toc2_2_)    
    - [¿Cómo utilizar los embeddings ?](#toc2_2_1_)    
    - [Check de los embeddings](#toc2_2_2_)    
  - [Convierte el texto a word embeddings](#toc2_3_)    
    - [**Tarea 1 (Question2Vec):**](#toc2_3_1_)    
  - [Evaluación de similitud de texto. Problemas de ranking / information retrieval](#toc2_4_)    
    - [Hits @ K](#toc2_4_1_)    
    - [DCG @ K](#toc2_4_2_)    
    - [Ejemplo de las métricas](#toc2_4_3_)    
    - [**Tasks 2 y 3 (HitsCount y DCGScore):**](#toc2_4_4_)    
  - [ Algoritmo: embeddings pre-entrenados](#toc2_5_)    
    - [**Tarea 4 (ranking)**](#toc2_5_1_)    
  - [Extra: SentenceTransformers y HuggingFace](#toc2_6_)    
  - [Extra: Efecto de preprocesado en modelos basados en Transformers](#toc2_7_)    
  - [Extra: Usa un modelo de fasttext en lugar de word2vec](#toc2_8_)    
  - [Extra (Difícil pero más interesante)](#toc2_9_)    
  - [Extra: Entrena tu propio word-embedding con gensim y usálo para crear un algoritmo de ranking](#toc2_10_)    
  - [Referencias y Lecturas Recomendadas](#toc2_11_)    
    - [Librerías](#toc2_11_1_)    
    - [Implementaciones de Sklearn](#toc2_11_2_)    
    - [Referencias](#toc2_11_3_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc2_'></a>[Encontrar preguntas similares en StackOverflow](#toc0_)

En este documento vamos a resolver un problema bastante común: encontrar preguntas duplicadas en un foro. En este caso, lo aplicaremos a los títulos de preguntas en [StackOverflow](https://stackoverflow.com).

La estrategia que usaremos se basa en algo mucho más general, que es el problema de encontrar dos textos que son similares semánticamente. Esto es la base de muchas otras técnicas como topic modelling, generación de features para clasificadores, o el archifamoso RAG (Retrieval Augmented Generation) que permite crear sistemas tipo ChatGPT sobre tus propios documentos.

![](https://raw.githubusercontent.com/UKPLab/sentence-transformers/master/docs/img/SemanticSearch.png)

De cara a medir la eficacia de nuestra herramienta, vamos a considerar el siguiente caso de uso para nuestro sistema:
* El un usuario escribe el título de una nueva pregunta
* En ese momento, se le muestra al usuario un conjunto de preguntas similares, para que compruebe que no ha sido escrita

El objetivo es que entre esas preguntas que se le muestren, estén duplicados de su pregunta en caso de que existan. Esto no es ni más ni menos que un problema de Ranking (similar al de Google).

La primera pregunta es, **¿cómo medimos la eficacia de nuestro sistema?**

### <a id='toc2_1_1_'></a>[Librerías](#toc0_)

- [Gensim](https://radimrehurek.com/gensim/)
- [Numpy](http://www.numpy.org) 
- [scikit-learn](http://scikit-learn.org/stable/index.html) 
- [Nltk](http://www.nltk.org)


**Es recomendable el uso de GPU para la segunda parte de este análisis.**

**ACTIVA LA GPU si estás en Colab**

In [1]:
# ejecuta esta celda sólo si estás en colab
# %pip install -U sentence-transformers gensim==4.3.2 scipy==1.12.0 tqdm nltk

# import nltk

# nltk.download("stopwords")

In [ ]:
# utils es un módulo utils.py en la misma carpeta, no una librería externa
from __future__ import annotations

import utils
import gensim
import numpy as np
from tqdm.auto import tqdm

/opt/anaconda3/envs/nlp_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### <a id='toc2_1_2_'></a>[Datos](#toc0_)

Lo primero es descargar los datos necesarios, que recomendamos guardar en `../embeddings`, es decir, una carpeta "embeddings" por encima de la carpeta del capítulo, así podremos usar los embeddings en futuros análisis.

Puedes descargarlo del enlace aportado de Google Drive, o directamente desde la api de gensim (seguramente más lento, pero más cómodo).

In [3]:
import gensim.downloader as api

path_to_embeddings = api.load("word2vec-google-news-300", return_path=True)

print(path_to_embeddings)

/Users/odremanferrer/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz


## <a id='toc2_2_'></a>[Word embeddings](#toc0_)

En la tarea vamos a usar word-embeddings:

 - [Word vectors Pre-entrenados](https://code.google.com/archive/p/word2vec/) de Google que fueron entrenados en una parte del dataset 'Google News' (con alreadedor de 100K millones de palabras). Es un modelo con vectores de 300 dimensiones, y un vocabulario de 3 millones de palabras. Están en el archivo `word2vec-google-news-300.gz`.
 - También utilizaremos sentence embeddings, que directamente dan el embedding de un texto, para ello usaremos la librería [SentenceTransformers](https://www.sbert.net/index.html).
 - [Extra]: Puedes entrenar un modelo en nuestros datos usando gensim.

Más que entrenar tus propios embeddings, es más fácil comenzar con embeddings pre-entrenados. Carga los vectores de Goggle pre-entrenados y cárguelos usando la función [gensim.models.KeyedVectors.load_word2vec_format](https://radimrehurek.com/gensim/models/keyedvectors.html) de gensim con el parámetro `binary = True`. Si el tamaño de los vectores es mayor que la memoria disponible, puede cargar solo una parte del vocabulario definiendo el parámetro `limit` (recomendado: 500000).

In [4]:
### ESCRIBE AQUÍ TU CÓDIGO
wv_embeddings = gensim.models.KeyedVectors.load_word2vec_format(
    path_to_embeddings, 
    binary=True,
    limit=500000  # Limitamos a 500k palabras para no consumir demasiada memoria
)
###

### <a id='toc2_2_1_'></a>[¿Cómo utilizar los embeddings ?](#toc0_)

Una vez que haya cargado las representaciones, asegúrate de poder acceder a ellas. Primero, puedes verificar si los embeddings cargados contienen una palabra:

    'word' in wv_embeddings
    
También, puedes obtener el embedding correspondiente accediendo como si fuera un diccionario:

    wv_embeddings['word']
 
### <a id='toc2_2_2_'></a>[Check de los embeddings](#toc0_)
 
Para evitar errores durante la primera parte, podemos verificar que los embeddings cargados son correctos. Llama a la función `check_embeddings()`, implementada a continuación, que ejecuta 3 pruebas:
1. Encuentra la palabra más similar para las palabras "positivas" y "negativas" proporcionadas.
2. Encuentra qué palabra de la lista dada no coincide con las demás.
3. Encuentra la palabra más similar a la dada.

En el caso correcto, la función devolverá la cadena *¡Todo correcto!*. De lo contrario, repasa el código anterior.

In [ ]:
print(utils.check_embeddings(wv_embeddings))

¡Todo correcto!


## <a id='toc2_3_'></a>[Convierte el texto a word embeddings](#toc0_)

### <a id='toc2_3_1_'></a>[**Tarea 1 (Question2Vec):**](#toc0_)

Por lo general, se usan embeddings para las palabras, pero para la tarea necesitamos crear una representación para toda la pregunta. Se podría hacer de diferentes maneras. En nuestro caso, usaremos una **media** de todos los vectores de palabras en la pregunta. Para ello hay que implementar la función `question_to_vec()`, que calcula la representación de preguntas descrita anteriormente. Esta función debería funcionar con el texto de entrada tal como está, sin ningún procesamiento previo.

Ten en cuenta que los word embeddings como Word2Vec no asignan ningún vector para las palabras Out of Vocabulary (OOV). En este caso, no tengas en cuenta esas palabras para calcular el embedding del texto. Si la pregunta no tiene ninguna palabra conocida, la función debería devolver un vector de ceros (pero del mismo tamaño que el resto).

In [6]:
def question_to_vec(
    question: str, embeddings: gensim.models.keyedvectors.KeyedVectors
) -> np.ndarray:
    """
    Transforms a string to an embedding by averaging word embeddings.
    If a word has no embedding, it is ignored.
    If no words have an embedding, a vector of zeros is returned.

    Parameters
    ----------
    question: str
    embeddings: gensim.models.keyedvectors.KeyedVectors

    Returns
    -------
    vector representation for the question
    """
    dim = embeddings.vector_size
    
    # Split question into words
    words = question.split()
    
    # Initialize vector accumulator
    question_vec = np.zeros(dim)
    
    # Count words with embeddings
    words_found = 0
    
    # Sum embeddings for each word that exists in the model
    for word in words:
        if word in embeddings:
            question_vec += embeddings[word]
            words_found += 1
    
    # If we found any words, compute average
    if words_found > 0:
        question_vec /= words_found
        
    return question_vec

Para comprobar que la implementación es correcta, corre `question_to_vec_tests`.

In [ ]:
print(utils.question_to_vec_tests(question_to_vec, wv_embeddings))

Test correctos.


## <a id='toc2_4_'></a>[Evaluación de similitud de texto. Problemas de ranking / information retrieval](#toc0_)

Ahora tenemos un método para crear una representación de cualquier frase y esto nos da una forma de resolver el problema. Primero, necesitamos comprobar qué tan bien funciona nuestra solución (los vectores de Google + `question_to_vec()`), y para ello lo primero que tenemos que determinar **es cómo evaluar la calidad de un modelo**.

Podemos esperar que, si usamos buenos *embeddings*, la **similitud** entre los vectores de las preguntas duplicadas debería ser menor que entre dos aleatorias. Recuerda que para medir la similitud semántica entre dos embeddings, se utiliza la *cosine similarity* o similitud del coseno. Ordenando según la **similitud del coseno**, para cada sentencia de query, ordenamos el resto de sentencias de la base de datos, y el objetivo es que las preguntas duplicadas estén lo más cerca posible. Además, lo esperado es que el resto de preguntas **no son parecidas**, es decir, son casos negativos.


Por ejemplo, si tenemos la pregunta _"Exceptions What really happens"_ y estamos seguros de que otra pregunta _"How does the catch keyword determine the type of exception that was thrown"_ es un duplicado. Pero nuestro modelo no lo sabe e intenta encontrar la mejor opción también entre preguntas como _"How Can I Make These Links Rotate in PHP"_ , _"NSLog array description not memory address"_ y _"PECL_HTTP not recognised php ubuntu"_. El objetivo del modelo es ordenar todas estas 4 preguntas (1 *positiva* y *R* = 3 *negativas*) de tal forma que la duplicada esté en la primera posición.


En general, lo normal no es que el mejor candidato esté en la primera posición, y no queremos una métrica tan rígida que sólo considere una respuesta como buena si el resultado correcto está en la primera posición. Se pueden utilizar métricas **más suaves**, en las que consideramos la posición en la que se encuentra el candidato. **Cuanto más alta, mejor.**

Supongamos que nos fijamos en los primeros $K$ elementos del ranking propuestos por el modelo, y $N$ es el número total de documentos que son duplicados, es decir, los **casos positivos** que hay en el dataset. Dos de las métricas más comunes son:


### <a id='toc2_4_1_'></a>[Hits @ K](#toc0_)

La primera métrica es simplemente el promedio de resultados correctos con *K* fijo:

$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [dup_i \in topK(q_i)]$$

![](https://cdn.prod.website-files.com/660ef16a9e0687d9cc27474a/662c4327f27ee08d3e4d4b47_657781b1f9c868e0cda088f6_precision_recall_k11.png)

donde $q_i$ es la i-ésima consulta, $dup_i$ es su duplicado, $topK(q_i)$ son las $K$ primeras propuestas proporcionadas por nuestro modelo y el operador **indicador** $[dup_i \in topK(q_i)]$ es igual a 1 si el duplicado $q_i$ está entre las primeras $K$ propuestas y 0 en caso contrario (para más detalles sobre el operador [aquí](https://en.wikipedia.org/wiki/Iverson_bracket).


### <a id='toc2_4_2_'></a>[DCG @ K](#toc0_)
La segunda propuesta tiene más en cuenta la posición del ranking de la respuesta correcta. Es una [métrica DCG](https://en.wikipedia.org/wiki/Discounted_cumulative_gain) simplificada:

$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank_{dup_i})}\cdot[rank_{dup_i} \le K] $$

donde $rank_{dup_i}$ es una posición del duplicado en el ranking para la consulta $ q_i $. Según esta métrica, el modelo obtiene una mayor recompensa por una posición más alta en el ranking. Si la respuesta no aparece en topK, la recompensa es cero.

### <a id='toc2_4_3_'></a>[Ejemplo de las métricas](#toc0_)

Vamos a calcular las métricas definidas sobre el ejemplo que hemos introducido antes. En este caso tenemos que $N$ = 1 (sólo hay un duplicado), y que lo correcto es que $q_1$ fuera _"How does the catch keyword determine the type of exception that was thrown"_. Vamos a imaginar que el modelo devuelve el siguiente ranking:
1. _"How Can I Make These Links Rotate in PHP"_
2. _"How does the catch keyword determine the type of exception that was thrown"_
3. _"NSLog array description not memory address"_
4. _"PECL_HTTP not recognised php ubuntu"_

Usando este ranking, calcula la métrica _Hits@K_ para $K$ = 1, 2, 4: 
 
- [K = 1] $\text{Hits@1} = \frac{1}{1}\sum_{i=1}^1 \, [dup_i \in top1(q_i)] = [dup_1 \in top1(q_1)] = 0$ porque la respuesta correcta no aparece en el _top1_.
- [K = 2] $\text{Hits@2} = \frac{1}{1}\sum_{i=1}^1 \, [dup_i \in top2(q_i)] = [dup_1 \in top2(q_1)] = 1$ porque $rank_{dup_1} = 2$.
- [K = 4] $\text{Hits@4} = \frac{1}{1}\sum_{i=1}^1 \, [dup_i \in top4(q_i)] = [dup_1 \in top4(q_1)] = 1$

Usando este ranking, calcula la métrica _DCG@K_ para $K$ = 1, 2, 4:

- [K = 1] $\text{DCG@1} = \frac{1}{1} \sum_{i=1}^1\frac{1}{\log_2(1+rank_{dup_i})}\cdot[rank_{dup_i} \le 1] = \frac{1}{\log_2(1+rank_{dup_i})}\cdot[rank_{dup_i} \le 1] = 0$ porque la respuesta correcta no aparece en el _top1_.
- [K = 2] $\text{DCG@2} = \frac{1}{1} \sum_{i=1}^1\frac{1}{\log_2(1+rank_{dup_i})}\cdot[rank_{dup_i} \le 2] = \frac{1}{\log_2{3}}$, porque $rank_{dup_1} = 2$.
- [K = 4] $\text{DCG@4} = \frac{1}{1} \sum_{i=1}^1\frac{1}{\log_2(1+rank_{dup_i})}\cdot[rank_{dup_i} \le 4] = \frac{1}{\log_2{3}}$.


### <a id='toc2_4_4_'></a>[**Tasks 2 y 3 (HitsCount y DCGScore):**](#toc0_)

Implementa las funciones `hits_count()` y `dcg_score()`, como se han descrito antes. Cada función tiene dos entradas, `dup_ranks` and `k`. Donde `dup_ranks` es la lista que contiene las posiciones en el ranking de los duplicados. Por ejemplo, en el ejemplo anterior la longitud de `dup_ranks` es $N$=1, y `dup_ranks = [2]`, puesto que la respuesta correcta está en la posición 2.

In [8]:
def hits_count(dup_ranks: list[int], k: int) -> float:
    """
    Compute the metric Hits@k for a list of duplicates' ranks.
    The metric is defined as a number of correct hits
    (i.e., of ranks less than k) divided by the number of total duplicates.
    This is, the percentage of duplicates found in top k.

    Parameters
    ----------
    dup_ranks: list[int]
        list of duplicates' ranks; one rank per question;
        length is a number of questions which we are looking for duplicates;
        rank is a number from 1 to len(candidates of the question);
        e.g. [2, 3] means that the first duplicate has the rank 2, and the second 3.
    k: int
        number of top-ranked elements (k in Hits@k metric)

    Returns
    -------
    float
        return Hits@k value for current ranking
    """
    # Count how many duplicates are ranked k or better
    hits = sum(1 for rank in dup_ranks if rank <= k)
    
    # Divide by total number of duplicates to get percentage
    return hits / len(dup_ranks)


def dcg_score(dup_ranks: list[int], k: int) -> float:
    """
    Compute the metric Discounted Cumulative Gain (DCG) for a list of duplicates' ranks.
    The metric is defined as:
    DCG@k = 1/|Q| * SUM(1/log2(1 + rank_i)), where rank_i <= k
    where |Q| is the number of duplicates in the database,
    rank_i is the rank of the i-th duplicate,
    and the sum is over all duplicates with rank_i <= k.

    Parameters
    ----------
    dup_ranks: list[int]
      list of duplicates' ranks; one rank per question;
               length is a number of questions which we are looking for duplicates;
               rank is a number from 1 to len(candidates of the question);
               e.g. [2, 3] means that the first duplicate has the rank 2, the second one — 3.
    k: int
        number of top-ranked elements (k in DCG@k metric)

    Returns
    -------
    float
        return DCG@k value for current ranking
    """
    # Total number of questions
    n_questions = len(dup_ranks)
    
    # Calculate DCG for each rank <= k
    dcg = sum(
        1 / np.log2(1 + rank) 
        for rank in dup_ranks 
        if rank <= k
    )
    
    # Normalize by number of questions
    return dcg / n_questions

Vamos a comprobar que está bien programado:

In [ ]:
print(utils.test_hits(hits_count))

Tests correctos.


In [10]:
def dcg_score(dup_ranks: list[int], k: int) -> float:
    """
    Compute the metric Discounted Cumulative Gain (DCG) for a list of duplicates' ranks.
    The metric is defined as:
    DCG@k = 1/|Q| * SUM(1/log2(1 + rank_i)), where rank_i <= k
    where |Q| is the number of duplicates in the database,
    rank_i is the rank of the i-th duplicate,
    and the sum is over all duplicates with rank_i <= k.

    Parameters
    ----------
    dup_ranks: list[int]
      list of duplicates' ranks; one rank per question;
               length is a number of questions which we are looking for duplicates;
               rank is a number from 1 to len(candidates of the question);
               e.g. [2, 3] means that the first duplicate has the rank 2, the second one — 3.
    k: int
        number of top-ranked elements (k in DCG@k metric)

    Returns
    -------
    float
        return DCG@k value for current ranking
    """
    # Total number of questions
    n_questions = len(dup_ranks)
    
    # Calculate DCG for each rank <= k
    dcg = sum(
        1 / np.log2(1 + rank) 
        for rank in dup_ranks 
        if rank <= k
    )
    
    # Normalize by number of questions
    return dcg / n_questions

In [ ]:
print(utils.test_dcg(dcg_score))

Tests correctos.


Una generalización del DCG score es su versión normalizada, que siempre es un número entre 0 y 1. El DCG puede ser mayor que 1.

In [12]:
def ndcg_score(dup_ranks: list[int], k: int, ignore_ties: bool = False) -> float:
    """
    Compute the metric Normalized Discounted Cumulative Gain (NDCG) for a list of duplicates' ranks.
    The metric is defined as:
    NDCG@k = DCG@k / IDCG@k, where IDCG@k is the Ideal DCG@k value.
    IDCG@k is the maximum possible DCG@k value for the case
    when all duplicates are ranked first.

    Parameters
    ----------
    dup_ranks: list[int]
        list of duplicates' ranks; one rank per question;
        length is a number of questions which we are looking for duplicates;
        rank is a number from 1 to len(candidates of the question);
        e.g. [2, 3] means that the first duplicate has the rank 2, the second one — 3.
    k: int
        number of top-ranked elements (k in DCG@k metric)
    ignore_ties: bool
        assume that there are no ties in dup_ranks, default is False

    Returns
    -------
    float
        return NDCG@k value for current ranking
    """
    dcg_score_ = dcg_score(dup_ranks, k)
    if ignore_ties:
        ideal_rank = np.arange(len(dup_ranks)) + 1
    else:
        ideal_rank = np.ones(len(dup_ranks))
    # we assume that there can be ties
    ideal_dcg_score = dcg_score(ideal_rank, k)
    return dcg_score_ / ideal_dcg_score

## <a id='toc2_5_'></a>[ Algoritmo: embeddings pre-entrenados](#toc0_)

Tenemos sólo un corpus: **test**. Cada entrada separada por '\t', y el formato es el siguiente:
 - *test* tiene las siguientes columnas: *question*, *similar question*, *negative example 1*, *negative example 2*, ... 

Es decir, tenemos la query, la pregunta similar, y un gran número de otras preguntas que NO son similares.

Lo ideal es que nuestro algoritmo diga que, de todas las posibles preguntas, la *similar question* es la más similar.


Vamos a leer el corpus de *test*, en `data/test_small.tsv`. Que usaremos para evaluar nuestra solución.

In [13]:
def read_corpus(filename: str) -> list[list[str]]:
    data = []
    for line in open(filename, encoding="utf-8"):
        data.append(line.strip().split("\t"))
    return data


test = read_corpus("data/test_small.tsv")
print(f"{len(test)=}")

len(test)=1500


Vamos a explorar algunos casos:

In [14]:
idx = 10
query = test[idx][0]
similar_question = test[idx][1]
negative_questions = test[idx][2:]

print(f"Query:\n\t{query}")
print(f"Similar question:\n\t{similar_question}")
print("Sample of Negative questions:")
for nq in negative_questions[:3]:
    print("\t" + nq)

Query:
	How to make the page not redirect in Rails
Similar question:
	Rails 3: undefined method `remote_form_for'
Sample of Negative questions:
	C# run C++ OpenGL application in Panel
	Excel VBA - Use an existing string in called sub
	How do I change/customize the icon for the pegman on Google Maps?


### <a id='toc2_5_1_'></a>[**Tarea 4 (ranking)**](#toc0_)

Ahora que tenemos como construir las *features*, y tenémos métricas para evaluar como de bueno es nuestro algoritmo de ranking, vamos a construir la función que nos hace el ranking.

Para comparar preguntas, usaremos la similaridad de coseno o *cosine similarity*. Tenemos que implementar la función `rank_candidates()`. La función debe retornar una lista ordenada de pares *(índice inicial en la lista de candidatos, candidato)*.

Por ejemplo, si la lista de candidatos es `[a,b,c]` y el más similar es `c`, después `a`, y luego `b`, entonces la función debe retornar `[(2, c), (0, a), (1, b)]`

**Ayuda:** Puedes utilizar la función `cosine_similarity()` de `sklearn.metrics.pairwise`. 

En cualquier caso, es preferible usar una versión vectorizada de la función `cosine_similarity()`, es decir, que no utilice bucles ni *list comprehension*. Sino el cómputo será demasiado lento. La función de sklearn cumple esto.

In [15]:
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd


def rank_candidates(
    question: str,
    candidates: list[str],
    embeddings: gensim.models.keyedvectors.KeyedVectors,
) -> list[tuple[int, str]]:
    """
    Rank the candidates for the question using cosine similarity.

    Parameters
    ----------
    question: str
        query question
    candidates: list[str]
        a list of strings (candidates) which we want to rank
    embeddings: gensim.models.keyedvectors.KeyedVectors
        word embeddings

    Returns
    -------
    list[tuple[int, str]]
        a list of pairs (initial position in the list, question)
    """
    # Convert question and candidates to vectors
    q_vec = question_to_vec(question, embeddings)
    cand_vecs = np.array([question_to_vec(c, embeddings) for c in candidates])
    
    # Reshape query vector to 2D array for cosine_similarity
    q_vec = q_vec.reshape(1, -1)
    
    # Calculate cosine similarities between query and all candidates
    similarities = cosine_similarity(q_vec, cand_vecs)[0]
    
    # Create list of (index, similarity, candidate) tuples
    ranked_list = list(enumerate(zip(similarities, candidates)))
    
    # Sort by similarity in descending order
    ranked_list.sort(key=lambda x: x[1][0], reverse=True)
    
    # Return list of (original_index, candidate) pairs
    return [(idx, cand) for idx, (sim, cand) in ranked_list]

In [16]:
rank_candidates(
    "I love pets",
    ["I love animals ", "My name is David", "This is a umbrella"],
    wv_embeddings,
)

[(0, 'I love animals '), (1, 'My name is David'), (2, 'This is a umbrella')]

In [17]:
rank_candidates(
    "I love pets",
    ["This is a umbrella", "My name is David", "I love animals "],
    wv_embeddings,
)

[(2, 'I love animals '), (1, 'My name is David'), (0, 'This is a umbrella')]

Comprobamos que es correcto:

In [ ]:
print(utils.test_rank_candidates(rank_candidates, wv_embeddings))

Test correctos.


Ahora comprobamos cómo de bueno es nuestro algoritmo. Ejecuta las siguientes celdas para obtener los resultados.

El cálculo de la similaridad entre vectores lleva su tiempo, y este cálculo puede llevar unos 2 minutos.

In [19]:
wv_ranking = []
for line in tqdm(test):
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

100%|██████████| 1500/1500 [00:36<00:00, 41.08it/s]


In [20]:
for k in [1, 5, 10, 100, 500, 1000]:
    print(
        "DCG@%4d: %.3f | Hits@%4d: %.3f"
        % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k))
    )

DCG@   1: 0.217 | Hits@   1: 0.217
DCG@   5: 0.269 | Hits@   5: 0.317
DCG@  10: 0.287 | Hits@  10: 0.371
DCG@ 100: 0.327 | Hits@ 100: 0.569
DCG@ 500: 0.360 | Hits@ 500: 0.827
DCG@1000: 0.378 | Hits@1000: 1.000


Si has hecho todos los pasos correctamente, debes estar un poco frustrado con los resultados obtenidos. Vamos a intentar entender por qué la calidad es tan baja.

Lo primero, antes de trabajar con los datos, hay que explorarlos para tener una idea de cómo son. Vamos a imprimir algunas de las preguntas del dataset:

In [21]:
for line in test[:3]:
    q, *examples = line
    print(q, *examples[:3])

How to print a binary heap tree without recursion? How do you best convert a recursive function to an iterative one? How can i use ng-model with directive in angular js flash: drawing and erasing
How to start PhoneStateListener programmatically? PhoneStateListener and service Java cast object[] to model WCF and What does this mean?
jQuery: Show a div2 when mousenter over div1 is over when hover on div1 depenting on if it is on div2 or not it should act differently How to run selenium in google app engine/cloud? Python Comparing two lists of strings for similarities


Como puedes ver, estamos tratando con datos sin procesar. Tienen singos de puntuación, caracteres especiales y letras en mayúsculas. Esto provoca que para muchas palabras no tengamos un *word embedding* correspondiente. Por ejemplo, para la palabra "grid?". 

Para resolver esto, vamos a utilizar la función `text_prepare()`, que preprocesa el texto. Puedes usar también cualquiera de las técnicas de preprocesamiento utilizadas anteriormente.

Ahora, prepara el corpus de test:

In [ ]:
from utils import text_prepare

prepared_test = []
for line in test:
    prepared_test.append([text_prepare(sentence) for sentence in line])

Ahora evaluamos los resultados sobre los datos preprocesados:

In [23]:
wv_prepared_ranking = []
for line in tqdm(prepared_test):
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings)
    wv_prepared_ranking.append([r[0] for r in ranks].index(0) + 1)

100%|██████████| 1500/1500 [00:33<00:00, 44.98it/s]


In [24]:
for k in [1, 5, 10, 100, 500, 1000]:
    print(
        "DCG@%4d: %.3f | Hits@%4d: %.3f"
        % (k, dcg_score(wv_prepared_ranking, k), k, hits_count(wv_prepared_ranking, k))
    )

DCG@   1: 0.327 | Hits@   1: 0.327
DCG@   5: 0.394 | Hits@   5: 0.454
DCG@  10: 0.413 | Hits@  10: 0.512
DCG@ 100: 0.446 | Hits@ 100: 0.675
DCG@ 500: 0.467 | Hits@ 500: 0.845
DCG@1000: 0.483 | Hits@1000: 1.000


In [40]:
#!pip install torch torchvision torchaudio
#!pip install transformers
#!pip install sentence-transformers
#!pip install faiss-cpu
#!pip install fasttext
#!pip install annoy
#!pip install scikit-learn
!pip install torch torchvision torchaudio sentence-transformers


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


## <a id='toc2_6_'></a>[Extra: SentenceTransformers y HuggingFace](#toc0_)

Hay muchos modelos diferentes que se pueden utilizar para entrenar embeddings. Enfoques similares a wordembeddings son FastText, StarSpace.
Los enfoques más recientes se basan en modelos tipo Transformers, como pueden ser ELMo, BERT y SentenceTransformers.

Este artículo ["The Illustrated BERT, ELMo, and co. (How NLP Cracked Transfer Learning)"](http://jalammar.github.io/illustrated-bert/), te guía a través de algunos de ellos, por ejemplo:

* ELMo \[[paper](https://arxiv.org/abs/1802.05365)\] (Embeddings from Language Models). Estos embeddings son funciones aprendidas de los estados internos de un modelo de lenguaje bidireccional profundo (biLM), que se entrena previamente en un gran corpus de texto. Proporciona representaciones profundas de palabras contextualizadas que modelan tanto las características complejas del uso de las palabras (por ejemplo, sintaxis y semántica), como la forma en que estos usos varían a través de contextos lingüísticos (es decir, para modelar la polisemia). [Aquí](https://tfhub.dev/google/elmo/1) hay un buen tutorial sobre cómo utilizar las embeddings ELMo con TF Hub.

* BERT \[[paper](https://arxiv.org/abs/1810.04805)\] (Bidirectional Encoder Representations from Transformers). Estas embeddings se basan en la arquitectura Transformer, que [ha sido introducida en 2017](https://arxiv.org/abs/1706.03762) y desde entonces se ha convertido en el modelo líder tanto en NLP como en otras áreas. También puedes leer sobre Transformers [en este blogpost](http://jalammar.github.io/illustrated-transformer/). Puede encontrar embeddings BERT preentrenadas en [el repositorio oficial de Google-Research](https://github.com/google-research/bert). Entrenar las embeddings BERT desde cero es costoso desde el punto de vista computacional, por lo que la gente suele afinar los embeddings de propósito general existentes.

En nuestro caso, usaremos uno de los modelos de la librería Sentence Transformers, que tiene es de fácil uso.

Ahondaremos en este tipo de modelos en los siguientes capítulos.

**IMPORTANTE: Una GPU será necesaria ahora, si no tienes en tu propio ordenador, utiliza una sesión de Google Colab activando la GPU**.

In [33]:
# check gpu is available
import torch

print(torch.cuda.is_available())

False


In [34]:
from sentence_transformers import SentenceTransformer

# paraphrase-MiniLM-L3-v2 is one of the fastest models
# https://www.sbert.net/docs/pretrained_models.html#sentence-embedding-models
model = SentenceTransformer("paraphrase-MiniLM-L3-v2")

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


In [35]:
# Sentences are encoded by calling model.encode()
# Sentences we want to encode. Example:
sentence = "This framework generates embeddings for each input sentence"
embedding = model.encode([sentence])
embedding.shape

(1, 384)

In [36]:
import numpy as np

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

def question_to_vec_tf(sentences: list[str]) -> np.ndarray:
    """Transforms a list of strings to embeddings by averaging word embeddings.
    It uses the sentence transformer model to encode the sentences.

    Parameters
    ----------
    sentences : list[str]
        list of strings

    Returns
    -------
    np.ndarray
        matrix with embeddings for each sentence
    """
    # Mover el modelo a MPS
    model.to(device)
    # Codificar las oraciones
    embeddings = model.encode(sentences, convert_to_numpy=True)
    return embeddings

# Modificamos rank_candidates_tf para usar MPS
def rank_candidates_tf(
    question: str, candidates: list[str], question_to_vec_tf: callable
) -> list[tuple[int, str]]:
    """Rank the candidates for the question using cosine similarity.
    Modified version to use the sentence transformer model.

    Parameters
    ----------
    question : str
        query question
    candidates : list[str]
        a list of strings (candidates) which we want to rank
    question_to_vec_tf : callable
        function that transforms a list of strings to embeddings

    Returns
    -------
    list[tuple[int, str]]
        a list of pairs (initial position in the list, question)
    """
    # Obtener embeddings usando MPS
    q_vec = np.array([question_to_vec_tf(question)])
    cand_vecs = question_to_vec_tf(candidates)
    
    # Calcular similitud coseno
    cosines = np.array(cosine_similarity(q_vec, cand_vecs)[0])

    merged_list = list(zip(cosines, range(len(candidates)), candidates))
    sorted_list = sorted(merged_list, key=lambda x: x[0], reverse=True)
    result = [(index, sentence) for score, index, sentence in sorted_list]
    return result

Using device: mps


In [37]:
# Cargamos el modelo con soporte MPS
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("paraphrase-MiniLM-L3-v2").to(device)

# Ahora podemos ejecutar el código de evaluación
wv_ranking_tf = []
for line in tqdm(test):
    q, *ex = line
    ranks = rank_candidates_tf(q, ex, question_to_vec_tf)
    wv_ranking_tf.append([r[0] for r in ranks].index(0) + 1)


100%|██████████| 1500/1500 [12:16<00:00,  2.04it/s]


In [38]:
# Imprimir resultados
for k in [1, 5, 10, 100, 500, 1000]:
    print(
        "DCG@%4d: %.3f | Hits@%4d: %.3f"
        % (k, dcg_score(wv_ranking_tf, k), k, hits_count(wv_ranking_tf, k))
    )

DCG@   1: 0.572 | Hits@   1: 0.572
DCG@   5: 0.653 | Hits@   5: 0.719
DCG@  10: 0.668 | Hits@  10: 0.767
DCG@ 100: 0.695 | Hits@ 100: 0.893
DCG@ 500: 0.706 | Hits@ 500: 0.976
DCG@1000: 0.708 | Hits@1000: 1.000


Algunas conclusiones:
* Este modelo es mucho más poderoso que los word-embeddings anteriores.
* En general, el preprocesado no ayuda en su performance!

## <a id='toc2_7_'></a>[Extra: Efecto de preprocesado en modelos basados en Transformers](#toc0_)

Escoge un modelo basado en Transformers, como "paraphrase-TinyBERT-L6-v2" y revisa las métricas preprocesando y sin preprocesar.

In [42]:
# Primero, cargamos el modelo TinyBERT
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("paraphrase-TinyBERT-L6-v2")

# Evaluamos con preprocesado
wv_ranking_prep_tf = []
for line in tqdm(prepared_test):
    q, *ex = line
    ranks = rank_candidates_tf(q, ex, question_to_vec_tf)
    wv_ranking_prep_tf.append([r[0] for r in ranks].index(0) + 1)


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
100%|██████████| 1500/1500 [16:39<00:00,  1.50it/s]


In [46]:
for k in [1, 5, 10, 100, 500, 1000]:
    print(
        "DCG@%4d: %.3f | Hits@%4d: %.3f"
        % (k, dcg_score(wv_ranking_prep_tf, k), k, hits_count(wv_ranking_prep_tf, k))
    )

DCG@   1: 0.547 | Hits@   1: 0.547
DCG@   5: 0.637 | Hits@   5: 0.710
DCG@  10: 0.658 | Hits@  10: 0.773
DCG@ 100: 0.685 | Hits@ 100: 0.903
DCG@ 500: 0.694 | Hits@ 500: 0.972
DCG@1000: 0.697 | Hits@1000: 1.000


## <a id='toc2_8_'></a>[Extra: Usa un modelo de fasttext en lugar de word2vec](#toc0_)

Para tener información sobre los modelos pre-entrenados disponibles, visita la siguiente web.

https://fasttext.cc/docs/en/crawl-vectors.html

Con la librería de Python `fasttext` podrás descargarlos de forma sencilla.

**Imprime las métricas DCG@K y Hits@K al igual que en los modelos anteriores**

In [45]:
# descargamos un modelo, lleva bastante tiempo
# url = "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz"
import sys

!{sys.executable} -m pip install fasttext

import fasttext.util

fasttext.util.download_model("en", if_exists="ignore")  # English

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


 (100.00%) [=================================================> ]                                                 ]>                                                  ]>                                                  ]>                                                  ]>                                                  ]>                                                  ]>                                                  ]>                                                  ]>                                                  ]>                                                  ]>                                                  ]>                                                  ]>                                                  ]>                                                  ]>                                                  ]>                                                  ]>                                                  ]>                                                  ]> 

'cc.en.300.bin'

In [47]:
# cargamos el modelo tipo fasttext en gensim
# la ventaja de cargar el modelo y no los vectores es que tenemos OOV (out-of-vocabulary) vectors
ft_model = gensim.models.fasttext.load_facebook_model("cc.en.300.bin.gz")
ft_embeddings = ft_model.wv

In [48]:
# Evaluamos con FastText
wv_ranking_ft = []
for line in tqdm(prepared_test):
    q, *ex = line
    ranks = rank_candidates(q, ex, ft_embeddings)
    wv_ranking_ft.append([r[0] for r in ranks].index(0) + 1)

# Mostramos los resultados
print("Resultados con FastText:")
for k in [1, 5, 10, 100, 500, 1000]:
    print(
        "DCG@%4d: %.3f | Hits@%4d: %.3f"
        % (k, dcg_score(wv_ranking_ft, k), k, hits_count(wv_ranking_ft, k))
    )


100%|██████████| 1500/1500 [00:41<00:00, 36.43it/s]

Resultados con FastText:
DCG@   1: 0.305 | Hits@   1: 0.305
DCG@   5: 0.369 | Hits@   5: 0.429
DCG@  10: 0.388 | Hits@  10: 0.487
DCG@ 100: 0.431 | Hits@ 100: 0.697
DCG@ 500: 0.456 | Hits@ 500: 0.893
DCG@1000: 0.468 | Hits@1000: 1.000


## <a id='toc2_9_'></a>[Extra (Difícil pero más interesante)](#toc0_)

Implementa, para nuestro caso de uso, la estrategia Retrieve + Rerank.
Esta viene explicada en la [documentación de SentenceTransformers](https://www.sbert.net/examples/applications/retrieve_rerank/README.html).

Y una implementación aplicada a buscar párrafos relevantes en Wikipedia está en el [siguiente notebook](https://colab.research.google.com/github/UKPLab/sentence-transformers/blob/master/examples/applications/retrieve_rerank/retrieve_rerank_simple_wikipedia.ipynb#scrollTo=UlArb7kqN3Re)

In [ ]:


from sentence_transformers import SentenceTransformer, util
import torch

# Cargamos el modelo
model = SentenceTransformer("paraphrase-MiniLM-L3-v2")

# Función para rankear candidatos
def rank_candidates_retrieve_rerank(question, candidates, top_k=100):
    # Codificamos la pregunta y los candidatos
    question_embedding = model.encode(question, convert_to_tensor=True)
    candidate_embeddings = model.encode(candidates, convert_to_tensor=True)
    
    # Calculamos similitudes
    similarities = util.pytorch_cos_sim(question_embedding, candidate_embeddings)[0]
    
    # Obtenemos los índices ordenados por similitud
    sorted_indices = torch.argsort(similarities, descending=True)
    
    # Creamos la lista de resultados
    results = [(idx.item(), candidates[idx.item()]) for idx in sorted_indices]
    
    return results

# Evaluamos el sistema
wv_ranking_retrieve_rerank = []
for line in tqdm(prepared_test):
    q, *ex = line
    # Rankeamos los candidatos
    ranked = rank_candidates_retrieve_rerank(q, ex)
    # Obtenemos la posición del candidato correcto (índice 0)
    position = [r[0] for r in ranked].index(0) + 1
    wv_ranking_retrieve_rerank.append(position)

# Mostramos los resultados
print("Resultados con Retrieve + Rerank:")
for k in [1, 5, 10, 100, 500, 1000]:
    print(
        "DCG@%4d: %.3f | Hits@%4d: %.3f"
        % (k, dcg_score(wv_ranking_retrieve_rerank, k), k, hits_count(wv_ranking_retrieve_rerank, k))
    )

 55%|█████▍    | 818/1500 [06:44<05:37,  2.02it/s]


KeyboardInterrupt: 

## <a id='toc2_10_'></a>[Extra: Entrena tu propio word-embedding con gensim y usálo para crear un algoritmo de ranking](#toc0_)

In [54]:
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
import nltk
from nltk.corpus import stopwords
import numpy as np

# Descargamos los stopwords de NLTK si no los tenemos
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Preprocesamos el texto para el entrenamiento
def preprocess_text(text):
    # Tokenizamos y convertimos a minúsculas
    tokens = simple_preprocess(text)
    # Eliminamos stopwords
    tokens = [token for token in tokens if token not in stop_words]
    return tokens

# Preparamos los datos de entrenamiento
training_data = []
for line in test:
    # Procesamos cada pregunta en la línea
    for question in line:
        tokens = preprocess_text(question)
        if tokens:  # Solo añadimos si hay tokens después del preprocesado
            training_data.append(tokens)

# Entrenamos el modelo Word2Vec
model = Word2Vec(
    sentences=training_data,
    vector_size=300,  # Tamaño del vector de embedding
    window=5,         # Ventana de contexto
    min_count=2,      # Frecuencia mínima de palabras
    workers=4,        # Número de workers para el entrenamiento
    epochs=10         # Número de épocas
)

# Guardamos el modelo
model.save("custom_word2vec.model")

# Evaluamos el modelo entrenado
wv_ranking_custom = []
for line in tqdm(prepared_test):
    q, *ex = line
    # Usamos el modelo personalizado para rankear
    ranks = rank_candidates(q, ex, model.wv)
    wv_ranking_custom.append([r[0] for r in ranks].index(0) + 1)

# Mostramos los resultados
print("Resultados con Word2Vec personalizado:")
for k in [1, 5, 10, 100, 500, 1000]:
    print(
        "DCG@%4d: %.3f | Hits@%4d: %.3f"
        % (k, dcg_score(wv_ranking_custom, k), k, hits_count(wv_ranking_custom, k))
    )

# Opcional: Analizamos algunas palabras similares para verificar la calidad del modelo
print("\nEjemplos de palabras similares:")
for word in ['python', 'java', 'database', 'error']:
    if word in model.wv:
        similar_words = model.wv.most_similar(word, topn=5)
        print(f"Palabras similares a '{word}':")
        for similar_word, score in similar_words:
            print(f"  {similar_word}: {score:.3f}")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/odremanferrer/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
100%|██████████| 1500/1500 [00:28<00:00, 52.34it/s]

Resultados con Word2Vec personalizado:
DCG@   1: 0.324 | Hits@   1: 0.324
DCG@   5: 0.421 | Hits@   5: 0.506
DCG@  10: 0.442 | Hits@  10: 0.574
DCG@ 100: 0.486 | Hits@ 100: 0.784
DCG@ 500: 0.507 | Hits@ 500: 0.946
DCG@1000: 0.512 | Hits@1000: 1.000

Ejemplos de palabras similares:
Palabras similares a 'python':
  file: 0.443
  java: 0.426
  php: 0.365
  string: 0.362
  android: 0.319
Palabras similares a 'java':
  python: 0.426
  android: 0.403
  file: 0.394
  php: 0.369
  error: 0.334
Palabras similares a 'database':
  data: 0.425
  file: 0.384
  table: 0.382
  query: 0.348
  sql: 0.336
Palabras similares a 'error':
  file: 0.476
  using: 0.421
  data: 0.368
  server: 0.363
  net: 0.351


## <a id='toc2_11_'></a>[Referencias y Lecturas Recomendadas](#toc0_)

* Benchmark sobre la tarea de Approximate Nearest Neighbors [ANN Benchmarks](https://ann-benchmarks.com/)


### <a id='toc2_11_1_'></a>[Librerías](#toc0_)
* Librería para entrenamiento y uso de word embeddings [Gensim](https://radimrehurek.com/gensim/). Es muy eficiente, pero a día de hoy está en modo mantenimiento. Para la parte de topic modelling un buen sustituto basado en transformers es [BERTopic](https://github.com/MaartenGr/BERTopic)
* Wrapper sobre gensim para optimizar la creación de sentence embeddings [Fast Sentence Embeddings](https://github.com/oborchers/Fast_Sentence_Embeddings)
* Librería para uso sencillo de modelos tanto WordEmbeddings como Transformers para crear Sentence Embeddings. Además, tiene un hub de modelos con un ranking. [SentenceTransformers](https://sbert.net/)
* Librería de Facebook para ANN: búsquedas eficientes y escalables en espacios de embeddings [Faiss](https://github.com/facebookresearch/faiss)
* Librería de Spotify para ANN: búsquedas eficientes y escalables en espacios de embeddings [Annoy](https://github.com/spotify/annoy)

### <a id='toc2_11_2_'></a>[Implementaciones de Sklearn](#toc0_)
* Acc@K https://scikit-learn.org/stable/modules/generated/sklearn.metrics.top_k_accuracy_score.html
* DCG@K: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.dcg_score.html
* NDCG@K: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.ndcg_score.html
* BallTree Implementation for ANN: https://scikit-learn.org/stable/modules/neighbors.html#ball-tree


### <a id='toc2_11_3_'></a>[Referencias](#toc0_)
* Guy Shani and Asela Gunawardana, "Evaluating Recommendation Systems", Recommender Systems Handbook, Springer, 2015
* Explanation of Ranking metrics from [Recommenders Library](https://github.com/recommenders-team/recommenders/blob/main/examples/03_evaluate/evaluation.ipynb)